In [ ]:
import torch
from torch.utils.data import TensorDataset,DataLoader
import pytorch_forecasting
import numpy as np
import os
from scipy import stats

os.chdir(os.getcwd())
fieldname = '_6hourly_20090101-20191231.npy'
x1_arr = np.load('data_wind/z1000'+fieldname) # geopotential height data (9*6 resolution)
x2_arr = np.load('data_wind/ua1000'+fieldname) # zonal wind data (9*6 resolution)
x3_arr = np.load('data_wind/va1000'+fieldname) # meridional wind data (9*6 resolution)

x1_arr = x1_arr[:,:6,:]
x2_arr = x2_arr[:,:6,:]
x3_arr = x3_arr[:,:6,:]

x1_arr_norm = stats.zscore(x1_arr) # normalize
x2_arr_norm = stats.zscore(x2_arr)
x3_arr_norm = stats.zscore(x3_arr)
y_arr = np.load('data_wind/stationwind_6hourly_20090101-20191231.npy') # rain data

tensor_x = torch.Tensor(np.stack([x1_arr_norm,x2_arr_norm,x3_arr_norm],axis=1)) # join z,z and pv data
#tensor_x = torch.Tensor(np.stack([x2_arr_norm,x3_arr_norm],axis=1)) # join z,z and pv data

tensor_y = torch.Tensor(y_arr)

forecast_dataset = TensorDataset(tensor_x,tensor_y) # creates a dataset based on tensors

train_size = int(np.ceil(0.8*len(forecast_dataset)))
train_set, val_set = torch.utils.data.random_split(forecast_dataset,[train_size,len(forecast_dataset)-train_size])

In [ ]:
x1_arr.shape

In [ ]:
training_dataloader = DataLoader(train_set,batch_size=300,shuffle=True)
valid_dataloader = DataLoader(val_set,batch_size=300)

In [ ]:
tensor_x.shape
for x,y in training_dataloader:
    print(x.shape,y.shape)
    break

### activate autoreload so any changes you make to dataloader.py, model.py are automatically imported

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from model_wind_nn import Net
import matplotlib.pyplot as plt

net = Net()

## the training:


In [ ]:
import torch.nn as nn
import torch.optim as optim
import tqdm

loss_func = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001) 

In [ ]:
def compute_accuracy_and_loss(dataloader,net):
    total = 0
    correct = 0
    loss = 0
    
    mag_bins = torch.Tensor([0,1.5,3.3,5.5,7.9,10.7,13.8,17.1,20.7]) # Beaufort scale
    
    if torch.cuda.is_available():
        net.cuda()
    net.eval()
    
    n_batches = 0
    with torch.no_grad():
        for x,y in dataloader:
            n_batches+=1
            
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            pred = net(x)
            
            loss+= loss_func(pred,y).item()
            
            obs_beau = torch.bucketize(y.norm(dim=1),mag_bins)
            pred_beau = torch.bucketize(pred.norm(dim=1),mag_bins)
            obs_deg = torch.atan(y[:,1]/y[:,0]) ; pred_deg = torch.atan(pred[:,1]/pred[:,0])
            err_deg = 180*(obs_deg-pred_deg)/np.pi
            err_deg = torch.abs((err_deg+180)%360-180)
            
            correct_mag = obs_beau == pred_beau
            correct_deg = err_deg<=20
            
            # an accurate prediction is where the magnitude is the same on the Beaufort scale, 
            # and the direction is off by 20 degrees or less

            correct_batch = np.logical_and(correct_mag,correct_deg)
            
            correct+=sum(correct_batch).item()
            total+=len(y)
            
    loss = loss/n_batches      
    return correct/total, loss

In [ ]:
compute_accuracy_and_loss(training_dataloader,net)

In [ ]:
compute_accuracy_and_loss(valid_dataloader,net)

In [ ]:
# test loss
pred = net(x)
mag_bins = torch.Tensor([0,1.5,3.3,5.5,7.9,10.7,13.8,17.1,20.7]) # Beaufort scale for wind speed

obs_beau = torch.bucketize(y.norm(dim=1),mag_bins)
pred_beau = torch.bucketize(pred.norm(dim=1),mag_bins)
obs_deg = torch.atan(y[:,1]/y[:,0]) ; pred_deg = torch.atan(pred[:,1]/pred[:,0])
err_deg = 180*(obs_deg-pred_deg)/np.pi
err_deg = torch.abs((err_deg+180)%360-180)

correct_mag = obs_beau == pred_beau
correct_deg = err_deg<=20

# an accurate prediction is where the magnitude is the same on the Beaufort scale, 
# and the direction is off by 20 degrees or less
correct_batch = np.logical_and(correct_mag,correct_deg)

sum(correct_deg)

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(obs_beau, pred_beau)
plt.imshow(cm)

In [ ]:
plt.hist(np.linalg.norm(y_arr,axis=1))

In [ ]:
if torch.cuda.is_available():
    net.cuda()

In [ ]:
n_epochs = 100

training_loss_vs_epoch = []
validation_loss_vs_epoch = []

training_acc_vs_epoch = []
validation_acc_vs_epoch = []

pbar = tqdm.tqdm( range(n_epochs) )

for epoch in pbar:
    
    loss,correct,total = 0,0,0
    
    if len(validation_loss_vs_epoch) > 1:
        pbar.set_description('val acc:'+'{0:.5f}'.format(validation_acc_vs_epoch[-1])+
                             ', train acc:'+'{0:.5f}'.format(training_acc_vs_epoch[-1]))
    
    net.train() # put the net into "training mode"
    for x,y in training_dataloader:
        
        optimizer.zero_grad() # zero the gradient
        
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        pred = net(x)
        
        cur_lossfunc = loss_func(pred,y)
        loss+= cur_lossfunc.item()
        
        cur_lossfunc.backward()
        optimizer.step()
    net.eval() #put the net into evaluation mode
    
    
    train_acc, train_loss = compute_accuracy_and_loss(training_dataloader,net)
    valid_acc, valid_loss =  compute_accuracy_and_loss(valid_dataloader,net)
         
    training_loss_vs_epoch.append( train_loss)    
    training_acc_vs_epoch.append( train_acc )
    
    validation_acc_vs_epoch.append(valid_acc)
    
    validation_loss_vs_epoch.append(valid_loss)
    
    #save the model if the validation loss has decreased
#    if len(validation_loss_vs_epoch)==1 or validation_loss_vs_epoch[-2] > validation_loss_vs_epoch[-1]:
#        torch.save(net.state_dict(), 'trained_model.pt')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,3))

ax[0].plot(training_loss_vs_epoch,label='training')
ax[0].plot(validation_loss_vs_epoch,label='validation')

ax[1].plot(training_acc_vs_epoch)
ax[1].plot(validation_acc_vs_epoch)

plt.show()

Nearest Neighbor Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(torch.flatten(train_set.dataset.tensors[0],1,-1),train_set.dataset.tensors[1])
pred_neigh = torch.Tensor(neigh.predict(torch.flatten(val_set.dataset.tensors[0],1,-1)))
y_neigh = val_set.dataset.tensors[1]

In [ ]:
correct_neigh = 0
total_neigh = 0
obs_beau_neigh = torch.bucketize(y_neigh.norm(dim=1),mag_bins)
pred_beau_neigh = torch.bucketize(pred_neigh.norm(dim=1),mag_bins)
obs_deg_neigh = torch.atan(y_neigh[:,1]/y_neigh[:,0]) ; pred_deg_neigh = torch.atan(pred_neigh[:,1]/pred_neigh[:,0])
err_deg_neigh = 180*(obs_deg_neigh-pred_deg_neigh)/np.pi
err_deg_neigh = torch.abs((err_deg_neigh+180)%360-180)

correct_mag_neigh = obs_beau_neigh == pred_beau_neigh
correct_deg_neigh = err_deg_neigh<=20

correct_batch_neigh = torch.logical_and(correct_mag_neigh,correct_deg_neigh)

total_neigh+=len(y_neigh)
sum(correct_batch_neigh)/total_neigh